# Импорт данных

In [ ]:
!apt-get update
!apt-get install -y unrar
!apt-get install -y mdbtools
!pip install openai
!pip install rarfile
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install gigachat

import asyncio
from gigachat import GigaChat
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import zipfile
import rarfile
from openai import OpenAI


from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,387 kB]
Get:14 https://r2u.stat.illinois.edu/ubuntu 

Просто загрузка датасета и создание Pandas dataframe

In [ ]:
zip_path = '/content/drive/MyDrive/train_dataset_train ржд каталог.zip'

# Путь, куда будет разархивирован файл
extract_path = '/content/train_dataset'

# Создаем директорию для разархивированных данных, если она не существует
os.makedirs(extract_path, exist_ok=True)

# Разархивируем ZIP-файл
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Файлы успешно разархивированы в {extract_path}")

rar_path = '/content/train_dataset/ржд 2/Справочник товаров.rar'

# Директория для извлечённых файлов
extract_path_rar = '/content/train_dataset/ржд_2_Справочник_товаров/'

# Создаём директорию для извлечённых данных, если она не существует
os.makedirs(extract_path_rar, exist_ok=True)

# Извлекаем RAR-архив
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(extract_path_rar)

print("RAR-файл успешно извлечён.")

accdb_path = '/content/train_dataset/ржд_2_Справочник_товаров/Справочник товаров.accdb'

# Проверка наличия файла
if os.path.exists(accdb_path):
    print("Файл .accdb найден.")
else:
    print("Файл .accdb не найден. Проверьте путь.")
    # Прекращаем выполнение, если файл не найден
    raise FileNotFoundError(f"Файл {accdb_path} не найден.")

# Шаг 5: Получение списка таблиц в базе данных
import subprocess

# Используем команду mdb-tables для получения списка таблиц
try:
    tables_output = subprocess.check_output(['mdb-tables', '-1', accdb_path]).decode('utf-8')
    table_names = tables_output.strip().split('\n')
    print("Найдены следующие таблицы:")
    for table in table_names:
        print(table)
except subprocess.CalledProcessError as e:
    print("Не удалось получить список таблиц. Возможно, формат .accdb не поддерживается.")
    print(e)
    # Прекращаем выполнение, если не удалось получить таблицы
    raise

# Шаг 6: Экспорт каждой таблицы в отдельный CSV-файл

# Директория для сохранения CSV-файлов
csv_dir = '/content/train_dataset/exported_csv/'
os.makedirs(csv_dir, exist_ok=True)

for table in table_names:
    # Убираем возможные пробелы в имени таблицы
    table_clean = table.strip()
    csv_output_path = os.path.join(csv_dir, f"{table_clean}.csv")

    # Экспортируем таблицу в CSV с помощью mdb-export
    export_command = f'mdb-export "{accdb_path}" "{table_clean}" > "{csv_output_path}"'
    os.system(export_command)

    # Проверка успешности экспорта
    if os.path.exists(csv_output_path):
        print(f"Таблица '{table_clean}' успешно экспортирована в '{csv_output_path}'")
    else:
        print(f"Не удалось экспортировать таблицу '{table_clean}'")

# Шаг 7: Импорт всех CSV-файлов в pandas DataFrame
import glob

# Путь ко всем CSV-файлам
csv_files = glob.glob(os.path.join(csv_dir, '*.csv'))

# Создаём словарь для хранения DataFrame всех таблиц
dataframes = {}

for csv_file in csv_files:
    table_name = os.path.splitext(os.path.basename(csv_file))[0]
    try:
        df = pd.read_csv(csv_file)
        dataframes[table_name] = df
        print(f"Таблица '{table_name}' успешно загружена в DataFrame.")
    except Exception as e:
        print(f"Ошибка при загрузке таблицы '{table_name}': {e}")


df_ED = pd.read_csv('/content/train_dataset/exported_csv/ED_IZM.csv')
df_GOST = pd.read_csv('/content/train_dataset/exported_csv/GOST.csv')
df_MTR = pd.read_csv('/content/train_dataset/exported_csv/MTR.csv')
df_OKPD = pd.read_csv('/content/train_dataset/exported_csv/OKPD_2.csv')

# получим датафреймы

Файлы успешно разархивированы в /content/train_dataset
RAR-файл успешно извлечён.
Файл .accdb найден.
Найдены следующие таблицы:
ED_IZM
GOST
MTR
OKPD_2
Таблица 'ED_IZM' успешно экспортирована в '/content/train_dataset/exported_csv/ED_IZM.csv'
Таблица 'GOST' успешно экспортирована в '/content/train_dataset/exported_csv/GOST.csv'
Таблица 'MTR' успешно экспортирована в '/content/train_dataset/exported_csv/MTR.csv'
Таблица 'OKPD_2' успешно экспортирована в '/content/train_dataset/exported_csv/OKPD_2.csv'
Таблица 'OKPD_2' успешно загружена в DataFrame.
Таблица 'ED_IZM' успешно загружена в DataFrame.
Таблица 'MTR' успешно загружена в DataFrame.
Таблица 'GOST' успешно загружена в DataFrame.


In [ ]:
# создадим отдельный df для работы в определенном ОКПД2 для тестов
df_x = df_MTR[df_MTR.ОКПД2 == '26.20.40.110']

# Работа с токенизацией, векторизацией и OHE

In [ ]:
df_x['text'] = df_x['Наименование'] + ' ' + df_x['Маркировка'] + ' ' + df_x['Параметры'] + ' ' + df_x['Регламенты (ГОСТ/ТУ)']
df_x['text'] = df_x['text'].astype(str)
df_x['text'] = df_x['text'].apply(lambda x: x.lower())

<ipython-input-6-a140da3a7d31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['text'] = df_x['Наименование'] + ' ' + df_x['Маркировка'] + ' ' + df_x['Параметры'] + ' ' + df_x['Регламенты (ГОСТ/ТУ)']
<ipython-input-6-a140da3a7d31>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['text'] = df_x['text'].astype(str)
<ipython-input-6-a140da3a7d31>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

Токенизация

In [ ]:
import spacy

# Загружаем модель spaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

def spacy_tokenize(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.is_alpha or token.like_num]
    return tokens

# Применяем токенизацию
df_x['tokens'] = df_x['text'].apply(spacy_tokenize)

from sklearn.feature_extraction.text import TfidfVectorizer

# Объединяем токены в строки для векторизации
df_x['token_str'] = df_x['tokens'].apply(lambda x: ' '.join(x))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_x['token_str'])

<ipython-input-7-8a64a88be2f7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['tokens'] = df_x['text'].apply(spacy_tokenize)
<ipython-input-7-8a64a88be2f7>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['token_str'] = df_x['tokens'].apply(lambda x: ' '.join(x))


Сделаем кластеризацию по knn

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df_x['cluster'] = kmeans.fit_predict(X)

<ipython-input-9-5abc51c24781>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['cluster'] = kmeans.fit_predict(X)


Рассмотрим кластеры:




In [ ]:
print('Кластер 0\n',
df_x[df_x.cluster == 0]['Наименование'].unique()[:5],
'\nКластер 1\n',
df_x[df_x.cluster == 1]['Наименование'].unique()[:5],
'\nКластер 2\n',
df_x[df_x.cluster == 2]['Наименование'].unique()[:5])

Кластер 0
 ['БЛОК ПИТАНИЯ' 'ФИЛЬТР СЕТЕВОЙ' 'ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'
 'БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА' 'ИСТОЧНИК ПИТАНИЯ БЕСПЕРЕБОЙНЫЙ'] 
Кластер 1
 ['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ' 'БЛОК ПИТАНИЯ'
 'БЛОК ЭЛЕКТРОПИТАНИЯ ИМПУЛЬСНЫЙ' 'ИСТОЧНИК РЕЗЕРВНОГО ЭЛЕКТРОПИТАНИЯ'
 'ИСТОЧНИК ВТОРИЧНОГО ЭЛЕКТРОПИТАНИЯ РЕЗЕРВИРОВАННЫЙ'] 
Кластер 2
 ['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ ДВОЙНОГО ПРЕОБРАЗОВАНИЯ (ОНЛАЙН) НАПОЛЬНОГО ИСПОЛНЕНИЯ БЕЗ ВСТРОЕННЫХ АКБ'
 'ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ ДВОЙНОГО ПРЕОБРАЗОВАНИЯ (ОНЛАЙН) НАПОЛЬНО-СТОЕЧНОГО ИСПОЛНЕНИЯ'
 'МОДУЛЬ БАТАРЕЙНЫЙ НАПОЛЬНО-СТОЕЧНОГО ИСПОЛНЕНИЯ ДЛЯ ИСТОЧНИКА БЕСПЕРЕБОЙНОГО ПИТАНИЯ'
 'БЛОК ПИТАНИЯ ДЛЯ ЩИТОВЫХ ЦИФРОВЫХ ПРИБОРОВ'
 'БАТАРЕЯ АККУМУЛЯТОРНАЯ СВИНЦОВО-КИСЛОТНАЯ']


Хоть данные и нормированы, knn работает плохо

Далее пробуем с OneHotEncoding (по словам) + с разной стандартизацией

В итоге попробовали все способы стандартизации


In [ ]:
# Применяем MultiLabelBinarizer для создания OneHot представления
mlb = MultiLabelBinarizer()
onehot_tokens = mlb.fit_transform(df_x['tokens'])

# Создаем новый DataFrame с OneHot токенами
df_onehot = pd.DataFrame(onehot_tokens, columns=mlb.classes_)

kmeans = KMeans(n_clusters=3, random_state=42)
df_x['cluster'] = kmeans.fit_predict(df_onehot)

print('СПОСОБ 1 БЕЗ СТАНДАРТИЗАЦИИ\n')
print('Кластер 0\n',
df_x[df_x.cluster == 0]['Наименование'].unique()[:10],
'\nКластер 1\n',
df_x[df_x.cluster == 1]['Наименование'].unique()[:10],
'\nКластер 2\n',
df_x[df_x.cluster == 2]['Наименование'].unique()[:10])

# Вычисляем среднее по каждому столбцу
means = df_onehot.mean(axis=0)

# Вычисляем стандартное отклонение по каждому столбцу
stds = df_onehot.std(axis=0)

# Применяем стандартизацию (z-score нормализацию)
df_onehot_standardized = (df_onehot - means) / stds

# Заполняем возможные NaN значения (если стандартное отклонение равно 0) нулями
df_onehot_standardized = df_onehot_standardized.fillna(0)

# Считаем количество положительных случаев (1) в каждом столбце
positive_counts = df_onehot.sum(axis=0)

# Домножаем стандартизированные значения на количество 1 в столбце
df_onehot_standardized_weighted = df_onehot_standardized.mul(positive_counts, axis=1)

df_onehot_standardized_weighted.head(2)

kmeans = KMeans(n_clusters=3, random_state=42)
df_x['cluster'] = kmeans.fit_predict(X)

print('\n\nСПОСОБ 1 БЕЗ СТАНДАРТИЗАЦИИ\n')

print('Кластер 0\n',
df_x[df_x.cluster == 0]['Наименование'].unique()[:10],
'\nКластер 1\n',
df_x[df_x.cluster == 1]['Наименование'].unique()[:10],
'\nКластер 2\n',
df_x[df_x.cluster == 2]['Наименование'].unique()[:10])

СПОСОБ 1 БЕЗ СТАНДАРТИЗАЦИИ

Кластер 0
 ['БЛОК ПИТАНИЯ' 'ФИЛЬТР СЕТЕВОЙ' 'ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'
 'БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА' 'ИСТОЧНИК ПИТАНИЯ БЕСПЕРЕБОЙНЫЙ'
 'МОДУЛЬ БАТАРЕЙНЫЙ БЕСПЕРЕБОЙНОГО ПИТАНИЯ'
 'БЛОК ПИТАНИЯ ИМПУЛЬСНЫЙ ГЕРМЕТИЧНЫЙ'
 'СИСТЕМА АВТОНОМНОГО ПИТАНИЯ ДЛЯ СВЕТИЛЬНИКА ПЕРЕНОСНОГО СИНЕГО СВЕТА'
 'МОДУЛЬ БЛОКА ПИТАНИЯ' 'БЛОК ПИТАНИЯ КОМПЬЮТЕРНОГО МОНОБЛОКА'] 
Кластер 1
 ['БАТАРЕЯ АККУМУЛЯТОРНАЯ СВИНЦОВО-КИСЛОТНАЯ'
 'АККУМУЛЯТОР СТАЦИОНАРНЫЙ СВИНЦОВО-КИСЛОТНЫЙ'] 
Кластер 2
 ['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ ДВОЙНОГО ПРЕОБРАЗОВАНИЯ (ОНЛАЙН) НАПОЛЬНОГО ИСПОЛНЕНИЯ БЕЗ ВСТРОЕННЫХ АКБ'
 'ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ ДВОЙНОГО ПРЕОБРАЗОВАНИЯ (ОНЛАЙН) НАПОЛЬНО-СТОЕЧНОГО ИСПОЛНЕНИЯ']


СПОСОБ 1 БЕЗ СТАНДАРТИЗАЦИИ

Кластер 0
 ['БЛОК ПИТАНИЯ' 'ФИЛЬТР СЕТЕВОЙ' 'ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'
 'БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА' 'ИСТОЧНИК ПИТАНИЯ БЕСПЕРЕБОЙНЫЙ'
 'МОДУЛЬ БАТАРЕЙНЫЙ БЕСПЕРЕБОЙНОГО ПИТАНИЯ'
 'БЛОК ПИТАНИЯ ИМПУЛЬСНЫЙ ГЕРМЕТИЧНЫЙ'
 'СИСТЕМА АВТОНОМНОГО ПИТ

<ipython-input-22-c32462058039>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['cluster'] = kmeans.fit_predict(df_onehot)
<ipython-input-22-c32462058039>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['cluster'] = kmeans.fit_predict(X)


**Итог работы с кластерами**

KNN находит определенные паттерны, но слабо. Нужно научиться регулировать важность слов для разбиения на кластеры.
Этот способ нам не подходит.
Будем использовать другие

# Разметка параметров LLM

Предварительно в другом ноутбуке мы сделали парсер товара с Яндекс Маркета. В нем мы парсили возможные параметры для типа товара. То есть на выходе в столбце параметры мы получили строку в которой перечислены все параметры, которые можно в теории извлечь из товара.

In [ ]:
# вот загрузка файла с итоговым парсером
df_x = pd.read_csv('/content/itog_data_from_pars.csv')

df_x.head(3)

,Unnamed: 0,код СКМТР,Наименование,Маркировка,Регламенты (ГОСТ/ТУ),Параметры,Базисная Единица измерения,ОКПД2,is_tu,ОКПД2_сокр,Параметры парс
0,248,4025240324,БЛОК ПИТАНИЯ,DEEPCOOL PF500,NaN,500ВТ ATX12V V2.4 ВЕНТИЛЯТОР=120Х120 86Х150Х14...,796,26.20.40.110,0,26,|форм-фактор|мощность|стандарт эффективности|pfc
1,963,4025201299,ФИЛЬТР СЕТЕВОЙ,TDM У05В,NaN,220В 5М 5РОЗЕТОК С ЗАЗЕМЛЕНИЕМ,796,26.20.40.110,0,26,|тип|длина кабеля|особенности|максимальная наг...
2,1177,4025201304,ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ,IPPON BACK POWER PRO II EURO 850,NaN,"850ВА 480ВТ, 2ХSCHUKO CEE 7, 100Х290Х140 5,5КГ...",796,26.20.40.110,0,26,|выходная мощность|выходных разъемов|разъемов ...


Тут нужен API к OpenAI (в целях конфиденциальности скрыт)

In [ ]:
client = OpenAI(api_key="")

def get_response(user_input: str, model: str = "gpt-4o-mini"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

sys_prompt = 'Для текста товара и его параметров определи свойства его, пиши в виде словаря питон. Пример ответа - {пол: "мужской"}. Название свойства - свойство. Все в нижнем регистре. Не пиши ничего дополнительно Все числа делай в одном формате всегда.(если у товара нет такого свойство или оно не написано - оставляй место пустым и не придумывай). Для всех строк пиши в одном формате. Там где тип товара - бери название из его наименования. Самое главное слово из наименования и 2 самых главных. Можно скзать группа. Вот  тебе список свойств по которым нужно работать: . '

dict_data_list = []
for i in tqdm(range(len(df_x))):
  user_prompt = ''
  user_prompt += str(df_x.iloc[i, 1]) + ' '
  user_prompt += str(df_x.iloc[i, 2]) + ' '
  user_prompt += str(df_x.iloc[i, 4]) + ' '
  parameters = str(df_x.iloc[i, 9])
  prompt = sys_prompt + parameters + '|Тип товара в одно слово|Тип товара в 2 слова| Больше ничего не надо' + 'Начало описаний товара' + user_prompt
  response = get_response(user_input=prompt)

  resp = response
  cleaned_resp = resp.replace('\\n', '').replace('\\', '')
  try:
    data_dict = json.loads(cleaned_resp)
    data_dict.setdefault('Наименование', []).append(df_x.iloc[i, 1])  # Добавляем значение из второго столбца (индекс 1)
    data_dict.setdefault('Параметры_изначальные', []).append(df_x.iloc[i, 4])
    dict_data_list.append(data_dict)
  except Exception as e:
    print(e)

df_gpt = pd.DataFrame()
for dict_data in dict_data_list:
    # Создаем DataFrame из словаря
    temp_df = pd.DataFrame([dict_data])
    # Объединяем с основным DataFrame
    df_gpt = pd.concat([df_gpt, temp_df], ignore_index=True)
df_gpt

После прогона этой модели получаем следующий датасет:

In [ ]:
df_gpt = pd.read_csv('/content/LLM_checked_df (1).csv')
df_gpt = df_gpt.drop(columns = ['Unnamed: 0', 'тип товара в 2 слова'])
df_gpt['тип товара в одно слово'] = df_gpt['тип товара в одно слово'].str.replace(' ', '', regex=False)
df_gpt

,форм-фактор,мощность,стандарт эффективности,pfc,тип товара в одно слово,Наименование,Параметры_изначальные,тип,длина кабеля,особенности,...,Тип товара в 2 слова,артикул маркета,тип товара в два слова,назначение,модель совместимого устройства,тип аккумулятора,цвет товара,выход. напряжение (мин),корпус,артикула маркета
0,atx,500,NaN,NaN,блок,['БЛОК ПИТАНИЯ'],['500ВТ ATX12V V2.4 ВЕНТИЛЯТОР=120Х120 86Х150Х...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,фильтр,['ФИЛЬТР СЕТЕВОЙ'],['220В 5М 5РОЗЕТОК С ЗАЗЕМЛЕНИЕМ'],фильтр,5м,с заземлением,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['850ВА 480ВТ, 2ХSCHUKO CEE 7, 100Х290Х140 5,5...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],['850ВА 425ВТ 1/5-290/220/230/240В 4ХIEC-320-C...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['100А, 4800ВТ, 267Х483Х314, 22КГ, ИСПОЛНЕНИЕ ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,atx,850,platinum,NaN,блок,['БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА'],['850ВТ 100-240В 12А ATX 12В V2.4 150Х160Х86'],NaN,NaN,NaN,...,NaN,NaN,блок питания,NaN,NaN,NaN,NaN,NaN,NaN,NaN
710,NaN,NaN,NaN,NaN,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['1000ВА/1000ВТ 12А 220/50В 88(2U)Х438Х420 15,...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711,NaN,NaN,NaN,NaN,модуль,['МОДУЛЬ БАТАРЕЙНЫЙ ИСТОЧНИКА БЕСПЕРЕБОЙНОГО П...,['36В (3Х12) 18АЧАС 88(2U)Х438Х520 24.9КГ'],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,NaN,NaN,NaN,NaN,источник,['ИСТОЧНИК ПИТАНИЯ НА DIN-РЕЙКУ'],['UВХ AC/DC=90-370В/UВЫХ=24В 5А 120ВТ UРЕГ=24-...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


На первый взгляд много Nan

Но суть в том, что разным типам товара соответствуют разные характеристики.

А сама разметка группы товара у нас сохранилась в столбце - "тип товара в одно слово"

Иногда может требоваться более подробный, но пока берем 1 группу.


У каждой группы получились свои характеристические свойства

In [ ]:
# Получаем уникальные значения из столбца 'тип товара в одно слово'
unique_types = df_gpt['тип товара в одно слово'].unique()

for product_type in unique_types:
    df_filtered = df_gpt[df_gpt['тип товара в одно слово'] == product_type]
    df_filtered = df_filtered.loc[:, df_filtered.isna().mean() < 0.9]
    globals()[f"df_{product_type}"] = df_filtered  # Создание отдельного DataFrame с названием

In [ ]:
df_блок

,форм-фактор,мощность,стандарт эффективности,тип товара в одно слово,Наименование,Параметры_изначальные
0,atx,500,NaN,блок,['БЛОК ПИТАНИЯ'],['500ВТ ATX12V V2.4 ВЕНТИЛЯТОР=120Х120 86Х150Х...
5,atx,400,NaN,блок,['БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА'],['400ВТ ATX DВЕНТИЛЯТОРА=120 1PCI-E(6-PIN) 2SA...
8,atx,500,NaN,блок,['БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА'],['500ВТ PFC ATX 12V 2.3 150Х86Х140']
12,NaN,850,80 plus bronze,блок,['БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА'],['850ВТ ATX 20+4ПИН 80 PLUS BRONZE ВЕНТИЛЯТОР ...
19,atx,450,NaN,блок,['БЛОК ПИТАНИЯ'],['450 ВТ']
...,...,...,...,...,...,...
703,NaN,5,NaN,блок,['БЛОК ПИТАНИЯ ДЛЯ СЕТЕВОГО ОБОРУДОВАНИЯ'],"['5В 5ВТ 1А ШТЕКЕР=5,5-2,5']"
704,NaN,120,NaN,блок,['БЛОК ПИТАНИЯ НА DIN-РЕЙКУ'],['120ВТ 12В 10А 63.5Х126Х125']
707,NaN,150,NaN,блок,['БЛОК ПИТАНИЯ ДЛЯ КОММУТАТОРА'],"['ВХОД U/100-240В 3А ВЫХОД U/12В 12,5А РЕГУЛИР..."
708,atx,850,gold,блок,['БЛОК ПИТАНИЯ СИСТЕМНОГО БЛОКА'],['850ВТ 100-240В 12А ATX 12В V2.4 150Х140Х86']


In [ ]:
df_источник

,тип товара в одно слово,Наименование,Параметры_изначальные,выходная мощность,выходных разъемов,интерфейсы
2,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['850ВА 480ВТ, 2ХSCHUKO CEE 7, 100Х290Х140 5,5...",480,2,NaN
3,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],['850ВА 425ВТ 1/5-290/220/230/240В 4ХIEC-320-C...,425вт,4,NaN
4,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['100А, 4800ВТ, 267Х483Х314, 22КГ, ИСПОЛНЕНИЕ ...",4800вт,NaN,NaN
6,источник,['ИСТОЧНИК ПИТАНИЯ БЕСПЕРЕБОЙНЫЙ'],['СТОЙКА/НАПОЛЬНЫЙ/1000ВА/900ВТ/USB/RS-232/SNM...,900,8,"usb, rs-232, snmp slot, epo"
9,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['1500ВА 900ВТ 49,5-60,5ГЦ 12В/9АЧАС 320Х125Х2...",900,NaN,usb
...,...,...,...,...,...,...
702,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['700 ВА, 390 ВТ, 6 X IEC 320 C13 (КОМПЬЮТЕРНА...",700,6,"rj-45, rs-232, usb type b"
706,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ ИМПУЛЬСНЫЙ Б...,"['130Х100Х40 0,41КГ']",NaN,NaN,NaN
710,источник,['ИСТОЧНИК БЕСПЕРЕБОЙНОГО ПИТАНИЯ'],"['1000ВА/1000ВТ 12А 220/50В 88(2U)Х438Х420 15,...",1000вт,NaN,NaN
712,источник,['ИСТОЧНИК ПИТАНИЯ НА DIN-РЕЙКУ'],['UВХ AC/DC=90-370В/UВЫХ=24В 5А 120ВТ UРЕГ=24-...,120,NaN,NaN


# Анализ разбиений

In [ ]:
import plotly.express as px
px.histogram(y = (1 - df_gpt.isna().mean().values) , x = df_gpt.isna().mean().index, template = 'simple_white',
             title = 'Доля заполненности признаков'
)